In [1]:
import pickle
import base64
import os
import glob
import hashlib
import globus_sdk

In [4]:
# Some variables
transfer_client = None
bookmarks = {}

# Where to get data
# Public shared data from Comet
comet_share = 'Cat Pictures on Comet'
comet_path = '/catpictures/'

# Where to send data 
# Server running Jupyter kernel
jh_demo = 'JupyterHub Demo'
jh_path = '/home/rpwagner/demodata/'

# Where to send results
# My laptop running Globus Connect Personal
my_laptop = 'RickMacBook'
my_laptop_path = '/Users/rpwagner/tmp/demodata/'

In [5]:
# Get the token from the notebook environment
data = pickle.loads(base64.b64decode(os.getenv('GLOBUS_DATA')))
print(data['tokens']['transfer.api.globus.org']['access_token'])
transfer_token = data['tokens']['transfer.api.globus.org']['access_token']

AQBZhjJXAAAAAAAFkWteZqH5KrTp4zReowUjzWCAqIbGVAUtmfvK4ewLQ27Sov3Yd1ntHzVBZiojxJpZmYeI


In [6]:
# Something that handles the token
authorizer = globus_sdk.AccessTokenAuthorizer(transfer_token)

# Something that can talk to Globus Transfer
transfer_client = globus_sdk.TransferClient(authorizer=authorizer)

# Grab my bookmarked endpooints
bookmark_data = transfer_client.bookmark_list()['DATA']

# Look at list of bookmarks
for bookmark in bookmark_data:
    bookmarks[bookmark['name']] = bookmark['endpoint_id']
    print(bookmark['name'], '\n    ', bookmark['endpoint_id'], '\n')

Tutorial 1 
     ddb59aef-6d04-11e5-ba46-22000b92c6ec 

RickMacBook 
     0062f30a-8ff0-11e6-b067-22000b92c261 

Comet 
     de463f97-6d04-11e5-ba46-22000b92c6ec 

JupyterHub Demo 
     985d6830-422d-11e7-bd35-22000b9a448b 

Cat Pictures on Comet 
     4950fae2-78d6-11e7-8bcb-22000b9923ef 



In [8]:
# What's in that Comet share?
ls_output = transfer_client.operation_ls(bookmarks[comet_share], path=comet_path)['DATA']
for thing in ls_output:
    print(thing['name'])

Chaton-corail.jpg
kitty_gauss08-c.png
kitty_gauss08.png


In [9]:
# Look at local directory
# Should be an empty directory
glob.glob(jh_path + '/*')

[]

In [10]:
# Thing to hold list of things to tranfer
# and source and destination endpoints
tdata = globus_sdk.TransferData(transfer_client, 
                                bookmarks[comet_share],
                               bookmarks[jh_demo])
# Add a files or folder
tdata.add_item(comet_path, jh_path, recursive=True)

# Kick off transfer
transfer_submission = transfer_client.submit_transfer(tdata)

In [18]:
# Neurotically check status (i.e., keep evaluating this cell)
task = transfer_client.get_task(transfer_submission['task_id'])
print(task['status'])

SUCCEEDED


In [19]:
# Look at local directory
# Should NOT be an empty directory
filenames = glob.glob(jh_path + '/*')
for filename in filenames:
    print(filename)

/home/rpwagner/demodata/kitty_gauss08-c.png
/home/rpwagner/demodata/Chaton-corail.jpg
/home/rpwagner/demodata/kitty_gauss08.png


In [20]:
# Pseudowork 
for filename in filenames:
    print(hashlib.md5(open(filename,'rb').read()).hexdigest())

1daaf5a3d476c3a68612acf0084cfbc8
a1de7d358906166449540d8ff53ae065
a838d27bbb0fcbd97f9b04099ee9524c


In [21]:
# Thing to hold list of things to tranfer
# and source and destination endpoints
tdata = globus_sdk.TransferData(transfer_client, 
                                bookmarks[jh_demo],
                               bookmarks[my_laptop])
# Add a files or folder
tdata.add_item(jh_path, my_laptop_path, recursive=True)

# Kick off transfer
transfer_submission = transfer_client.submit_transfer(tdata)

In [28]:
# Neurotically check status (i.e., keep evaluating this cell)
task = transfer_client.get_task(transfer_submission['task_id'])
print(task['status'])

SUCCEEDED
